In [2]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from networks.model import TD3_Actor
from environment.env import TradingEnv
from pykalman import KalmanFilter
from sklearn.preprocessing import MinMaxScaler
from environment.wrappers import env_wrapper
import matplotlib.pyplot as plt
from scipy.special import softmax

g:\Code\Python\GitHub\Portfolio-Optimizer\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
for i in range(5):
    print(i)

0
1
2
3
4


In [15]:
test = pd.read_csv(os.path.join('data', 'Close.csv'), index_col=0, parse_dates=True).values.T

In [25]:
test = np.expand_dims(test, axis=0)

In [26]:
test = np.concatenate([test, test, test], axis=0)

In [35]:
test_a = test + np.random.normal(loc=0, scale=0.25, size=test.shape)

In [34]:
test

array([[[7.02000000e+03, 6.54320422e+03, 6.47577864e+03, ...,
         4.64680000e+04, 4.71235754e+04, 4.62039329e+04],
        [3.14500000e+02, 2.97697323e+02, 3.13990000e+02, ...,
         3.62968997e+03, 3.70931457e+03, 3.67749619e+03],
        [1.00818002e+00, 1.00601006e+00, 1.00934994e+00, ...,
         1.00029397e+00, 1.00103605e+00, 1.00018895e+00],
        ...,
        [1.22729919e+02, 1.22748436e+02, 1.22813271e+02, ...,
         2.41845779e+02, 2.41457001e+02, 2.40689392e+02],
        [2.19004868e+02, 2.18875580e+02, 2.19134094e+02, ...,
         4.37536377e+02, 4.36300537e+02, 4.35104584e+02],
        [1.49083817e+02, 1.49074142e+02, 1.49258438e+02, ...,
         4.01054169e+02, 3.99855652e+02, 3.97358765e+02]],

       [[7.02000000e+03, 6.54320422e+03, 6.47577864e+03, ...,
         4.64680000e+04, 4.71235754e+04, 4.62039329e+04],
        [3.14500000e+02, 2.97697323e+02, 3.13990000e+02, ...,
         3.62968997e+03, 3.70931457e+03, 3.67749619e+03],
        [1.00818002e+00, 

In [36]:
test_a

array([[[7.01972791e+03, 6.54329378e+03, 6.47581178e+03, ...,
         4.64678338e+04, 4.71233788e+04, 4.62040382e+04],
        [3.13916436e+02, 2.98051288e+02, 3.13378072e+02, ...,
         3.62954026e+03, 3.70916872e+03, 3.67712954e+03],
        [1.02794304e+00, 1.13311759e+00, 9.71298810e-01, ...,
         8.39908245e-01, 7.48825032e-01, 5.82664803e-01],
        ...,
        [1.22310138e+02, 1.22713293e+02, 1.22505160e+02, ...,
         2.42088210e+02, 2.41351280e+02, 2.40824590e+02],
        [2.18848244e+02, 2.18665448e+02, 2.19103364e+02, ...,
         4.37705279e+02, 4.36529747e+02, 4.35424265e+02],
        [1.49090123e+02, 1.48844590e+02, 1.48815153e+02, ...,
         4.01084500e+02, 3.99626255e+02, 3.97471026e+02]],

       [[7.02009344e+03, 6.54314266e+03, 6.47600658e+03, ...,
         4.64679101e+04, 4.71240259e+04, 4.62038790e+04],
        [3.14377363e+02, 2.98288712e+02, 3.13949311e+02, ...,
         3.62937641e+03, 3.70883170e+03, 3.67705937e+03],
        [1.06418406e+00, 

In [8]:
test

array([[7.02000000e+03, 6.54320422e+03, 6.47577864e+03, ...,
        4.64680000e+04, 4.71235754e+04, 4.62039329e+04],
       [3.14500000e+02, 2.97697323e+02, 3.13990000e+02, ...,
        3.62968997e+03, 3.70931457e+03, 3.67749619e+03],
       [1.00818002e+00, 1.00601006e+00, 1.00934994e+00, ...,
        1.00029397e+00, 1.00103605e+00, 1.00018895e+00],
       ...,
       [1.22729919e+02, 1.22748436e+02, 1.22813271e+02, ...,
        2.41845779e+02, 2.41457001e+02, 2.40689392e+02],
       [2.19004868e+02, 2.18875580e+02, 2.19134094e+02, ...,
        4.37536377e+02, 4.36300537e+02, 4.35104584e+02],
       [1.49083817e+02, 1.49074142e+02, 1.49258438e+02, ...,
        4.01054169e+02, 3.99855652e+02, 3.97358765e+02]])

In [20]:
test / test[0, -1]

array([[[0.79401137, 1.1442475 , 1.88476997, 2.9387764 ],
        [0.55904318, 1.29927868, 1.69766974, 0.26191503],
        [1.30524131, 0.15427702, 0.16971658, 1.93649807],
        [1.04078574, 1.39597516, 1.08558159, 1.43830296],
        [0.1835339 , 0.95206068, 1.349897  , 0.04074164],
        [0.45299555, 0.93542293, 3.32338026, 2.97546933],
        [1.34851473, 0.20500576, 3.59920244, 0.05944025],
        [1.        , 1.        , 1.        , 1.        ]],

       [[0.27728008, 0.34926733, 1.44382683, 2.12008267],
        [0.84206811, 0.3909853 , 3.08937013, 0.66746445],
        [0.97945481, 0.49459849, 3.18511958, 0.47003919],
        [1.40719988, 0.3335148 , 3.54189653, 2.68201261],
        [0.36243357, 0.37792133, 3.6381877 , 2.69962016],
        [0.35661596, 0.44442165, 2.00202057, 0.55289329],
        [0.06653715, 0.0432124 , 3.48046896, 0.39913318],
        [0.86817136, 0.98861233, 0.11586043, 0.89937261]],

       [[0.80005926, 0.8551943 , 0.04574103, 0.54809041],
        [1

In [2]:
env = TradingEnv('data', commission=0.01, steps=750, start_date_index=0)
env = env_wrapper(env)

In [9]:
env.step(np.array([0, 10, 2, 0, 55, 0, 0, 0]))[0]['action']

array([4.27230414e-05, 9.41037611e-01, 3.15682950e-04, 3.84507373e-04,
       5.70659539e-02, 3.84507373e-04, 3.84507373e-04, 3.84507373e-04])

In [33]:
a = softmax(np.array([0, 0, 0, 0, 0, 0, 0, 0]))
a = np.concatenate([a[:3]*0.1, a[3:]*0.9])
(a / a.sum())

array([0.02083333, 0.02083333, 0.02083333, 0.1875    , 0.1875    ,
       0.1875    , 0.1875    , 0.1875    ])

array([0.01840864, 0.01132587, 0.01046812, 0.12739284, 0.06305695,
       0.14455485, 0.14063896, 0.06253275])